In [45]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import heapq as heap

In [46]:
flights = pd.read_csv('Flight_Data.csv')

In [47]:
df = flights.drop_duplicates(subset=['SourceAirport', 'DestinationAirport', 'SourceAirport_City', 'SourceAirport_Country',
                               'SourceAirport_Latitude', 'SourceAirport_Longitude', 'SourceAirport_Altitude',
                               'DestinationAirport_City', 'DestinationAirport_Country', 'DestinationAirport_Latitude',
                               'DestinationAirport_Longitude', 'DestinationAirport_Altitude', 'Distance', 'FlyTime', 'Price'])


In [48]:
df = df.reset_index(drop = True)

In [49]:
df

,Airline,SourceAirport,DestinationAirport,SourceAirport_City,SourceAirport_Country,SourceAirport_Latitude,SourceAirport_Longitude,SourceAirport_Altitude,DestinationAirport_City,DestinationAirport_Country,DestinationAirport_Latitude,DestinationAirport_Longitude,DestinationAirport_Altitude,Distance,FlyTime,Price
0,Pegasus Airlines,Sabiha Gökçen International Airport,Imam Khomeini International Airport,Istanbul,Turkey,40.898602,29.309200,312,Tehran,Iran,35.416100,51.152199,3305,1998.541333,2.982660,969.270667
1,Turkish Airlines,Atatürk International Airport,Imam Khomeini International Airport,Istanbul,Turkey,40.976898,28.814600,163,Tehran,Iran,35.416100,51.152199,3305,2040.978811,3.035377,990.489405
2,Emirates,Dubai International Airport,Imam Khomeini International Airport,Dubai,United Arab Emirates,25.252800,55.364399,62,Tehran,Iran,35.416100,51.152199,3305,1199.863567,1.990514,569.931784
3,Etihad Airways,Abu Dhabi International Airport,Imam Khomeini International Airport,Abu Dhabi,United Arab Emirates,24.433001,54.651100,88,Tehran,Iran,35.416100,51.152199,3305,1266.681453,2.073517,603.340726
4,Air Arabia,Sharjah International Airport,Imam Khomeini International Airport,Sharjah,United Arab Emirates,25.328600,55.517200,111,Tehran,Iran,35.416100,51.152199,3305,1196.902147,1.986835,568.451073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6831,Kenmore Air,Boeing Field King County International Airport,William R Fairchild International Airport,Seattle,United States,47.529999,-122.302002,21,Port Angeles,United States,48.120201,-123.500000,291,110.930397,0.637802,25.465199
6832,Kenmore Air,Boeing Field King County International Airport,Orcas Island Airport,Seattle,United States,47.529999,-122.302002,21,Eastsound,United States,48.708199,-122.910004,31,138.564899,0.672130,39.282450
6833,Kenmore Air,William R Fairchild International Airport,Boeing Field King County International Airport,Port Angeles,United States,48.120201,-123.500000,291,Seattle,United States,47.529999,-122.302002,21,110.930397,0.637802,25.465199
6834,Kenmore Air,Friday Harbor Airport,Boeing Field King County International Airport,Friday Harbor,United States,48.521999,-123.024002,113,Seattle,United States,47.529999,-122.302002,21,122.677708,0.652395,31.338854


### Airline                             
### اسم خط هوایی 

### SourceAirport 
### فرودگاه مبدا 

### DestinationAirport  
### فرودگاه مقصد

### SourceAirport_City
### شهر فرودگاه مبدا

### SourceAirport_Country 
### کشور فرودگاه مبدا

### SourceAirport_Latitude          
### عرض جغرافیایی فرودگاه مبدا

### SourceAirport_Longitude                  
### طول جغرافیایی فرودگاه مبدا

### SourceAirport_Altitude            
### ارتفاع جغرافیایی فرودگاه مبدا

### DestinationAirport_City    
### شهر فرودگاه مقصد

### DestinationAirport_Country 
### کشور فرودگاه مقصد

### DestinationAirport_Latitude          
### عرض جغرافیایی فرودگاه مقصد

### DestinationAirport_Longitude            
### طول جغرافیایی فرودگاه مقصد

### DestinationAirport_Altitude       
### ارتفاع جغرافیایی فرودگاه مقصد

### Distance               
### فاصله

### FlyTime               
### زمان پرواز

### Price          
### هزینه

In [50]:
df['SourceAirport_Location'] = df.apply(lambda row: (row['SourceAirport_Latitude'], row['SourceAirport_Longitude'], row['SourceAirport_Altitude']), axis=1)

# Drop the individual columns since you now have a single combined column
df.drop(['SourceAirport_Latitude', 'SourceAirport_Longitude', 'SourceAirport_Altitude'], axis=1, inplace=True)


In [51]:
# Create a new column 'DestinationAirport_Location' by combining the latitude, longitude, and altitude into a tuple
df['DestinationAirport_Location'] = df.apply(lambda row: (row['DestinationAirport_Latitude'], row['DestinationAirport_Longitude'], row['DestinationAirport_Altitude']), axis=1)

# Drop the individual columns since you now have a single combined column
df.drop(['DestinationAirport_Latitude', 'DestinationAirport_Longitude', 'DestinationAirport_Altitude'], axis=1, inplace=True)


In [52]:
def haversine(lat1, lon1, alt1, lat2, lon2, alt2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Earth radius in meters (you can use different values based on your dataset)
    R = 6371000

    # Altitude difference
    alt_diff = abs(alt1 - alt2)

    # Haversine formula
    a = math.sin((lat2 - lat1) / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin((lon2 - lon1) / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Calculate the distance
    distance = R * c + alt_diff

    return distance

In [53]:
class Node:
    def __init__(self, airport, city, country, coordination):
        self.airport = airport
        self.city = city
        self.country = country
        self.coordination = coordination


In [54]:
class Edge:
    def __init__(self, airline, distance, fly_time, price):
        self.airline = airline
        self.distance = distance
        self.fly_time = fly_time
        self.price = price


In [55]:
df

,Airline,SourceAirport,DestinationAirport,SourceAirport_City,SourceAirport_Country,DestinationAirport_City,DestinationAirport_Country,Distance,FlyTime,Price,SourceAirport_Location,DestinationAirport_Location
0,Pegasus Airlines,Sabiha Gökçen International Airport,Imam Khomeini International Airport,Istanbul,Turkey,Tehran,Iran,1998.541333,2.982660,969.270667,"(40.898601532, 29.3092002869, 312)","(35.416099548339844, 51.15219879150391, 3305)"
1,Turkish Airlines,Atatürk International Airport,Imam Khomeini International Airport,Istanbul,Turkey,Tehran,Iran,2040.978811,3.035377,990.489405,"(40.976898, 28.8146, 163)","(35.416099548339844, 51.15219879150391, 3305)"
2,Emirates,Dubai International Airport,Imam Khomeini International Airport,Dubai,United Arab Emirates,Tehran,Iran,1199.863567,1.990514,569.931784,"(25.2527999878, 55.3643989563, 62)","(35.416099548339844, 51.15219879150391, 3305)"
3,Etihad Airways,Abu Dhabi International Airport,Imam Khomeini International Airport,Abu Dhabi,United Arab Emirates,Tehran,Iran,1266.681453,2.073517,603.340726,"(24.433000564575195, 54.651100158691406, 88)","(35.416099548339844, 51.15219879150391, 3305)"
4,Air Arabia,Sharjah International Airport,Imam Khomeini International Airport,Sharjah,United Arab Emirates,Tehran,Iran,1196.902147,1.986835,568.451073,"(25.32859992980957, 55.5172004699707, 111)","(35.416099548339844, 51.15219879150391, 3305)"
...,...,...,...,...,...,...,...,...,...,...,...,...
6831,Kenmore Air,Boeing Field King County International Airport,William R Fairchild International Airport,Seattle,United States,Port Angeles,United States,110.930397,0.637802,25.465199,"(47.52999877929688, -122.302001953125, 21)","(48.12020111083984, -123.5, 291)"
6832,Kenmore Air,Boeing Field King County International Airport,Orcas Island Airport,Seattle,United States,Eastsound,United States,138.564899,0.672130,39.282450,"(47.52999877929688, -122.302001953125, 21)","(48.7081985474, -122.910003662, 31)"
6833,Kenmore Air,William R Fairchild International Airport,Boeing Field King County International Airport,Port Angeles,United States,Seattle,United States,110.930397,0.637802,25.465199,"(48.12020111083984, -123.5, 291)","(47.52999877929688, -122.302001953125, 21)"
6834,Kenmore Air,Friday Harbor Airport,Boeing Field King County International Airport,Friday Harbor,United States,Seattle,United States,122.677708,0.652395,31.338854,"(48.5219993591, -123.024002075, 113)","(47.52999877929688, -122.302001953125, 21)"


In [56]:
dft = df[['Distance', 'FlyTime', 'Price']]

In [57]:
dft

,Distance,FlyTime,Price
0,1998.541333,2.982660,969.270667
1,2040.978811,3.035377,990.489405
2,1199.863567,1.990514,569.931784
3,1266.681453,2.073517,603.340726
4,1196.902147,1.986835,568.451073
...,...,...,...
6831,110.930397,0.637802,25.465199
6832,138.564899,0.672130,39.282450
6833,110.930397,0.637802,25.465199
6834,122.677708,0.652395,31.338854


In [58]:
dft.corr()

,Distance,FlyTime,Price
Distance,1.0,1.0,1.0
FlyTime,1.0,1.0,1.0
Price,1.0,1.0,1.0


In [59]:
nodes = {}
graph = {}

In [60]:
for data in df.values:
    if data[1] not in nodes:
        nodes[data[1]] = Node(data[1], data[3], data[4], data[10])
    
    if data[2] not in nodes:
        nodes[data[2]] = Node(data[2], data[5], data[6], data[11])
    
    e = Edge(data[0], data[7], data[8], data[9])
    
    if data[1] not in graph:
        graph[data[1]] = {}
    
    if data[2] not in graph:
        graph[data[2]] = {}
        
    graph[data[1]][data[2]] = e

In [61]:
nodes

{'Sabiha Gökçen International Airport': <__main__.Node at 0x7fe6a47c9450>,
 'Imam Khomeini International Airport': <__main__.Node at 0x7fe6a85cb010>,
 'Atatürk International Airport': <__main__.Node at 0x7fe6a8465cd0>,
 'Dubai International Airport': <__main__.Node at 0x7fe6a4fab490>,
 'Abu Dhabi International Airport': <__main__.Node at 0x7fe6a7219c50>,
 'Sharjah International Airport': <__main__.Node at 0x7fe6a7218ad0>,
 'Hamad International Airport': <__main__.Node at 0x7fe6a7219090>,
 'Esenboğa International Airport': <__main__.Node at 0x7fe6a7218e90>,
 'Adnan Menderes International Airport': <__main__.Node at 0x7fe6a7218950>,
 "Chicago O'Hare International Airport": <__main__.Node at 0x7fe6a7219cd0>,
 'Los Angeles International Airport': <__main__.Node at 0x7fe6a7218a10>,
 'John F Kennedy International Airport': <__main__.Node at 0x7fe6a7218250>,
 'Antalya International Airport': <__main__.Node at 0x7fe6a7218d50>,
 'Adana Airport': <__main__.Node at 0x7fe6a7218dd0>,
 'Milas Bodrum

In [62]:
len(nodes)

759

In [63]:
graph

{'Sabiha Gökçen International Airport': {'Imam Khomeini International Airport': <__main__.Edge at 0x7fe6a8403750>,
  'Sharjah International Airport': <__main__.Edge at 0x7fe6a721b950>,
  'Dubai International Airport': <__main__.Edge at 0x7fe6a70dab10>,
  'Trabzon International Airport': <__main__.Edge at 0x7fe6a70db550>,
  'Antalya International Airport': <__main__.Edge at 0x7fe6a70dbf90>,
  'Adnan Menderes International Airport': <__main__.Edge at 0x7fe6a70d4150>,
  'Kayseri Erkilet Airport': <__main__.Edge at 0x7fe6a70d4510>,
  'Esenboğa International Airport': <__main__.Edge at 0x7fe6a70d45d0>,
  'Konya Airport': <__main__.Edge at 0x7fe6a70d4e50>,
  'Milas Bodrum International Airport': <__main__.Edge at 0x7fe6a70d4ed0>,
  'Adana Airport': <__main__.Edge at 0x7fe6a70d4f90>,
  'Batman Airport': <__main__.Edge at 0x7fe6a70d5150>,
  'Diyarbakir Airport': <__main__.Edge at 0x7fe6a70d5210>,
  'Dalaman International Airport': <__main__.Edge at 0x7fe6a70d5310>,
  'Çardak Airport': <__main_

In [104]:
def dijkstra_score(e : Edge, w1 = 1, w2 = 700, w3 = 2):
    return e.distance * w1 + e.fly_time * w2 + e.price * w3

In [105]:
def dijkstra(src, dest):
    pq = []
    heap.heappush(pq, (0, src))
    
    parent = {}
    dist = {}
    dist[src] = 0
    parent[src] = -1
    
    while pq:
            d, u = heap.heappop(pq)
            for v in graph[u]:
                weight = dijkstra_score(graph[u][v])
                if v not in dist:
                    dist[v] = dist[u] + weight
                    heap.heappush(pq, (dist[v], v))
                    parent[v] = u
                    
                elif dist[v] > dist[u] + weight:
                    dist[v] = dist[u] + weight
                    heap.heappush(pq, (dist[v], v))
                    parent[v] = u
    
    path = []
    start = dest
    distance = 0
    fly_time = 0
    price = 0
    
    while start != -1:
        path.append(start)
        finish = parent[start]
        
        if finish == -1:
            break
            
        e = graph[finish][start]
        
        distance += e.distance
        fly_time += e.fly_time
        price += e.price
        
        start = finish
        
    path = list(reversed(path))
    
    return (dist[dest], path, distance, fly_time, price)

In [106]:
print(dijkstra('Imam Khomeini International Airport', 'Raleigh Durham International Airport'))

(31801.692263190693, ['Imam Khomeini International Airport', 'Atatürk International Airport', 'John F Kennedy International Airport', 'Raleigh Durham International Airport'], 10779.22609171797, 14.890342971078221, 5299.613045858985)
